In [1]:
pip install --upgrade gradio

   ---------------------------------------- 0.0/59.7 MB ? eta -:--:--
   ---------------------------------------- 0.3/59.7 MB ? eta -:--:--
    --------------------------------------- 1.0/59.7 MB 5.6 MB/s eta 0:00:11
   - -------------------------------------- 2.6/59.7 MB 5.6 MB/s eta 0:00:11
   -- ------------------------------------- 4.2/59.7 MB 6.3 MB/s eta 0:00:09
   --- ------------------------------------ 5.5/59.7 MB 6.0 MB/s eta 0:00:10
   ----- ---------------------------------- 8.1/59.7 MB 7.4 MB/s eta 0:00:07
   ----- ---------------------------------- 8.9/59.7 MB 7.6 MB/s eta 0:00:07
   -------- ------------------------------- 12.1/59.7 MB 7.9 MB/s eta 0:00:07
   --------- ------------------------------ 13.9/59.7 MB 8.0 MB/s eta 0:00:06
   ---------- ----------------------------- 15.5/59.7 MB 7.9 MB/s eta 0:00:06
   ----------- ---------------------------- 17.0/59.7 MB 7.9 MB/s eta 0:00:06
   ------------ --------------------------- 18.9/59.7 MB 7.9 MB/s eta 0:00:06
   -----

In [9]:
pip install imageio-ffmpeg


   ---------------------------------------- 0.0/31.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/31.2 MB ? eta -:--:--
    --------------------------------------- 0.5/31.2 MB 1.5 MB/s eta 0:00:21
   - -------------------------------------- 0.8/31.2 MB 1.1 MB/s eta 0:00:29
   -- ------------------------------------- 1.6/31.2 MB 2.1 MB/s eta 0:00:14
   ---- ----------------------------------- 3.1/31.2 MB 3.2 MB/s eta 0:00:09
   ------ --------------------------------- 5.0/31.2 MB 4.3 MB/s eta 0:00:07
   -------- ------------------------------- 6.3/31.2 MB 4.5 MB/s eta 0:00:06
   -------- ------------------------------- 6.8/31.2 MB 4.5 MB/s eta 0:00:06
   --------- ------------------------------ 7.3/31.2 MB 4.1 MB/s eta 0:00:06
   ----------- ---------------------------- 9.2/31.2 MB 4.6 MB/s eta 0:00:05
   ------------ --------------------------- 10.0/31.2 MB 4.7 MB/s eta 0:00:05
   ------------- -------------------------- 10.7/31.2 MB 4.4 MB/s eta 0:00:05
   --------

In [2]:
!pip install -q imageio-ffmpeg


In [3]:
import imageio_ffmpeg
ffmpeg_bin = imageio_ffmpeg.get_ffmpeg_exe()
print("FFmpeg exe:", ffmpeg_bin)


FFmpeg exe: C:\Users\yalda\anaconda3\envs\cyclegan_env\lib\site-packages\imageio_ffmpeg\binaries\ffmpeg-win-x86_64-v7.1.exe


In [4]:
!pip install -q imageio-ffmpeg


In [1]:
import os
os.environ.setdefault("TORCHDYNAMO_DISABLE", "1")  
os.environ.setdefault("PYTORCH_JIT", "0")          

import sys, cv2, tempfile, numpy as np, subprocess, shutil, time, traceback
from pathlib import Path
from PIL import Image, ImageEnhance
import gradio as gr
import torch
from torchvision import transforms
import imageio_ffmpeg  

try:
    import torch._dynamo as dynamo
    dynamo.config.suppress_errors = True
    dynamo.disable()  
except Exception:
    pass

REPO_DIR = r"C:\Users\yalda\OneDrive\Desktop\Thesis apps\Project Folder\pytorch-CycleGAN-and-pix2pix"
os.chdir(REPO_DIR)
sys.path.append(REPO_DIR)

from options.test_options import TestOptions
from models import create_model


class JupyterTestOptions(TestOptions):
    def parse(self, dataroot="./datasets/dummy", model="pix2pix"):
        saved_argv = sys.argv[:]
        try:
            sys.argv = [saved_argv[0], "--dataroot", dataroot, "--model", model]
            opt = super().parse()
        finally:
            sys.argv = saved_argv
        opt.isTrain = False
        return opt

def _find_generator_checkpoint(ckpt_dir: str, run_name: str):
    run_dir = Path(ckpt_dir) / run_name
    if not run_dir.exists():
        return None, f"Run folder not found: {run_dir}"
    candidates = list(run_dir.glob("*_net_G.pth"))
    if not candidates:
        return None, f"No *_net_G.pth found in {run_dir}. Make sure your pix2pix weights are there."
    latest = next((p for p in candidates if p.name == "latest_net_G.pth"), candidates[0])
    return latest, None

def load_pix2pix(checkpoints_dir, run_name, size=512, gpu_preferred=True, direction="AtoB"):
    g_path, err = _find_generator_checkpoint(checkpoints_dir, run_name)
    if err:
        raise gr.Error(err)

    opt = JupyterTestOptions().parse(dataroot="./datasets/dummy", model="pix2pix")
    opt.isTrain = False
    opt.model = 'pix2pix'
    opt.dataroot = './datasets/dummy'
    opt.checkpoints_dir = checkpoints_dir
    opt.name = run_name
    opt.serial_batches = True
    opt.no_flip = True
    opt.batch_size = 1
    opt.num_threads = 0
    opt.load_size = size
    opt.crop_size = size
    opt.preprocess = 'none'
    opt.direction = direction
    opt.dataset_mode = 'aligned'
    opt.gpu_ids = [0] if (gpu_preferred and torch.cuda.is_available()) else [-1]

    model = create_model(opt)
    model.setup(opt)
    model.eval()
    return model, model.device, str(g_path)


def make_transform(size):
    return transforms.Compose([
        transforms.Resize((size, size)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

def tensor_to_uint8_img(t):
    if t.dim() == 4:
        t = t[0]
    img = t.detach().cpu().numpy()
    img = (np.transpose(img, (1,2,0)) + 1.0) / 2.0
    img = np.clip(img * 255.0, 0, 255).astype(np.uint8)
    return img

def post_random_lighting(bgr, brightness_jitter=0.03, contrast_jitter=0.03):
    pil = Image.fromarray(cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB))
    if brightness_jitter > 0:
        factor_b = np.random.uniform(1.0 - brightness_jitter, 1.0 + brightness_jitter)
        pil = ImageEnhance.Brightness(pil).enhance(factor_b)
    if contrast_jitter > 0:
        factor_c = np.random.uniform(1.0 - contrast_jitter, 1.0 + contrast_jitter)
        pil = ImageEnhance.Contrast(pil).enhance(factor_c)
    return cv2.cvtColor(np.array(pil), cv2.COLOR_RGB2BGR)

def _even(x):
    x = int(x)
    return x if x % 2 == 0 else x - 1

def _ensure_path_from_gradio_file(v):
    if v is None:
        return None
    if isinstance(v, (str, os.PathLike)):
        return str(v)
    if isinstance(v, dict):
        for k in ("name", "path", "orig_name"):
            val = v.get(k)
            if isinstance(val, (str, os.PathLike)) and os.path.exists(val):
                return str(val)
        for k in ("file", "video", "data"):
            val = v.get(k)
            if isinstance(val, dict):
                for kk in ("name", "path", "orig_name"):
                    vv = val.get(kk)
                    if isinstance(vv, (str, os.PathLike)) and os.path.exists(vv):
                        return str(vv)
    if hasattr(v, "read"):
        tmp = Path(tempfile.mkdtemp()) / "upload.mp4"
        with open(tmp, "wb") as f:
            f.write(v.read())
        return str(tmp)
    return None

def _open_writer(path: Path, fps: float, size_xy):
    w, h = size_xy
    fourccs = ['mp4v', 'avc1', 'H264', 'XVID']
    for four in fourccs:
        writer = cv2.VideoWriter(str(path), cv2.VideoWriter_fourcc(*four), fps, (w, h))
        if writer.isOpened():
            return writer, four
    return None, None


def process_video(
    video_path, checkpoints_dir, run_name, size=512,
    hold_k=2, brightness_jitter=0.03, contrast_jitter=0.03,
    gpu_preferred=True, progress: gr.Progress = gr.Progress(track_tqdm=False),
    direction="AtoB"
):
    if not os.path.exists(video_path):
        raise gr.Error(f"File not found: {video_path}")

    ffmpeg_bin = imageio_ffmpeg.get_ffmpeg_exe()
    if not ffmpeg_bin or not os.path.exists(ffmpeg_bin):
        raise gr.Error("FFmpeg binary not found from imageio_ffmpeg. Please reinstall imageio-ffmpeg.")

    model, device, g_path = load_pix2pix(checkpoints_dir, run_name, size=size, gpu_preferred=gpu_preferred, direction=direction)
    tfm = make_transform(size)

    workdir = Path(tempfile.mkdtemp(prefix="p2p_"))
    raw_video_path   = workdir / "comparison_raw.mp4"
    h264_video_path  = workdir / "comparison_h264.mp4"

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise gr.Error(
            "Could not open the video. "
            "Please upload an H.264 MP4. If your file is .mov/HEVC, re-encode to H.264 and try again."
        )

    fps = cap.get(cv2.CAP_PROP_FPS) or 24.0
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
    out_w, out_h = _even(size*2), _even(size)

    writer, used_fourcc = _open_writer(raw_video_path, fps, (out_w, out_h))
    if writer is None:
        cap.release()
        raise gr.Error("Failed to open VideoWriter for any common codecs (mp4v/avc1/H264/XVID).")

    last_processed = None
    frame_index = 0

    progress(0.0, desc="Processing video...")
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        left = cv2.resize(frame, (size, size), interpolation=cv2.INTER_AREA)

        do_infer = (frame_index % max(1, hold_k) == 0) or (last_processed is None)
        if do_infer:
            img_rgb = cv2.cvtColor(left, cv2.COLOR_BGR2RGB)
            A = tfm(Image.fromarray(img_rgb)).unsqueeze(0).to(device)
            with torch.no_grad():
                model.set_input({'A': A, 'B': A, 'A_paths': '', 'B_paths': ''})
                model.test()
                visuals = model.get_current_visuals()
                if 'fake_B' not in visuals:
                    cap.release(); writer.release()
                    raise gr.Error("Model did not return 'fake_B'. Check your checkpoint/model type/direction.")
                out_rgb = tensor_to_uint8_img(visuals['fake_B'])
                right = cv2.cvtColor(out_rgb, cv2.COLOR_RGB2BGR)
        else:
            right = last_processed.copy()

        right = post_random_lighting(right, brightness_jitter, contrast_jitter)
        sbs = np.hstack([cv2.resize(left, (size, size)), cv2.resize(right, (size, size))])
        sbs = cv2.resize(sbs, (out_w, out_h))
        writer.write(sbs)
        last_processed = right

        frame_index += 1
        if frame_index % 30 == 0:
            frac = (frame_index / total_frames) if total_frames > 0 else 0.0
            progress(frac, desc=f"Frames processed: {frame_index}")

    cap.release()
    writer.release()

    cmd = [
        ffmpeg_bin, "-y",
        "-i", str(raw_video_path),
        "-vcodec", "libx264",
        "-pix_fmt", "yuv420p",
        "-crf", "23",
        str(h264_video_path),
    ]
    try:
        subprocess.check_call(cmd)
    except subprocess.CalledProcessError as e:
        raise gr.Error(f"FFmpeg re-encode failed: {e}")

    out_dir = Path("gradio_outputs"); out_dir.mkdir(exist_ok=True)
    final_path = out_dir / f"output_{int(time.time())}.mp4"
    shutil.copyfile(h264_video_path, final_path)

    progress(1.0, desc=f"Done. Frames: {frame_index}")
    return str(final_path)


def ui_infer(video_file, checkpoints_dir, run_name, size, hold_k, brightness_jitter, contrast_jitter, use_gpu, progress=gr.Progress()):
    try:
        video_path = _ensure_path_from_gradio_file(video_file)
        if not video_path or not os.path.exists(video_path):
            raise gr.Error(f"Please upload a valid video file. Got: {repr(video_path)}")

        return process_video(
            video_path, checkpoints_dir, run_name, size=size,
            hold_k=hold_k, brightness_jitter=brightness_jitter,
            contrast_jitter=contrast_jitter, gpu_preferred=use_gpu, progress=progress,
            direction="AtoB"  
        )
    except gr.Error:
        raise
    except Exception as e:
        tb = traceback.format_exc()
        raise gr.Error(f"Unhandled error:\n{e}\n\nTraceback:\n{tb}")


with gr.Blocks(title="Stop-Motion Pix2Pix") as demo:
    gr.Markdown("### Stop-Motion Style Transfer (Pix2Pix)")

    video_file = gr.File(label="Upload Video", file_types=["video"], file_count="single")
    checkpoints_dir = gr.Textbox(value="./checkpoints", label="Checkpoints dir")
    run_name = gr.Textbox(value="coraline_p2p_v3_512", label="Run name")
    size = gr.Dropdown(choices=[256, 512, 1024], value=512, label="Model Size")
    use_gpu = gr.Checkbox(value=True, label="Use GPU")

    hold_k = gr.Slider(1, 5, value=2, step=1, label="Frame Hold (K)")
    brightness_jitter = gr.Slider(0.0, 0.15, value=0.03, step=0.01, label="Brightness Jitter")
    contrast_jitter = gr.Slider(0.0, 0.15, value=0.03, step=0.01, label="Contrast Jitter")

    run_btn = gr.Button("Run")
    out_video = gr.Video(label="Before | After", format="mp4")

    run_btn.click(
        ui_infer,
        inputs=[video_file, checkpoints_dir, run_name, size, hold_k, brightness_jitter, contrast_jitter, use_gpu],
        outputs=[out_video]
    )

demo.queue(max_size=8)
demo.launch(share=False, debug=True, allowed_paths=["."])


C:\Users\yalda\anaconda3\envs\cyclegan_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: ./datasets/dummy              	[default: None]
             dataset_mode: aligned                       
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
                load_iter: 0                            

In [6]:
pip install charset-normalizer

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install chardet


Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install ipywidgets jupyter nbextension enable --py widgetsnbextension


Note: you may need to restart the kernel to use updated packages.



Usage:   
  C:\Users\yalda\anaconda3\envs\cyclegan_env\python.exe -m pip install [options] <requirement specifier> [package-index-options] ...
  C:\Users\yalda\anaconda3\envs\cyclegan_env\python.exe -m pip install [options] -r <requirements file> [package-index-options] ...
  C:\Users\yalda\anaconda3\envs\cyclegan_env\python.exe -m pip install [options] [-e] <vcs project url> ...
  C:\Users\yalda\anaconda3\envs\cyclegan_env\python.exe -m pip install [options] [-e] <local project path> ...
  C:\Users\yalda\anaconda3\envs\cyclegan_env\python.exe -m pip install [options] <archive url/path> ...

ambiguous option: --py (--pypi-url, --python, --python-version?)


In [8]:
def compile_networks(self):
    import sys, torch
    if sys.platform.startswith("win") or not hasattr(torch, "compile"):
        return


In [10]:
import os
os.environ["TORCH_COMPILE_DISABLE"] = "1"


In [11]:
import torch, sys, platform, os
def _identity_compile(fn, *args, **kwargs):
    return fn
if hasattr(torch, "compile"):
    torch.compile = _identity_compile

print("Python:", sys.executable)
print("Torch:", torch.__version__, "CUDA:", torch.version.cuda)
print("Torch path:", os.path.dirname(torch.__file__))
print("Platform:", platform.system())


Python: C:\Users\yalda\anaconda3\envs\cyclegan_env\python.exe
Torch: 2.5.1+cu121 CUDA: 12.1
Torch path: C:\Users\yalda\anaconda3\envs\cyclegan_env\lib\site-packages\torch
Platform: Windows
